In [13]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
X_spam = pd.read_csv('spambase/spambase.data', header=None)
X_sp, y_sp = X_spam.iloc[:,:X_spam.shape[1]-1], X_spam.iloc[:,-1]
X_sp = X_sp.values
y_sp = y_sp.values
X_train, X_test, y_train, y_test = train_test_split(X_sp, y_sp, test_size=0.10, random_state=42)

In [15]:
print(X_train.shape)


(4140, 57)


In [16]:
# Training Parameters
sess.close()
learning_rate = 0.01
num_steps = 5000
batch_size = 100

display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 45 # 1st layer num features
num_hidden_2 = 4 # 2nd layer num features (the latent dim)
num_input = 57 # 

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}

In [17]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [18]:
def next_batch(data,num):

    #Return a total of `num` random samples and labels. 

    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]

    return data_shuffle

In [19]:
# # Start Training
# # Start a new TF session
# sess = tf.Session()

# # Run the initializer
# sess.run(init)

# # Training
# for i in range(1, num_steps+1):
#     # Prepare Data
#     # Get the next batch of MNIST data (only images are needed, not labels)
#     batch_x = next_batch(X_train,batch_size)

#     # Run optimization op (backprop) and cost op (to get loss value)
#     _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
#     # Display logs per step
#     if i % display_step == 0 or i == 1:
#         print('Step %i: Minibatch Loss: %f' % (i, l))

In [20]:
#sess.close()
sess = tf.Session()
sess.run(init)

hm_epoch = 40

for epoch in range(hm_epoch):
    l_total = 0
    
    for i in range(int(X_train.shape[0]/batch_size)):
        #_,l = sess.run([optimizer,loss],feed_dict={X: tNGX_train[i*batch_size:(i+1)*batch_size].toarray()})
        _,l = sess.run([optimizer,loss],feed_dict={X: next_batch(X_train,batch_size)})
        
        l_total += l
    print("The loss at : ",epoch," is : ",l_total)

The loss at :  0  is :  324252.3566894531
The loss at :  1  is :  327958.888671875
The loss at :  2  is :  271668.4608154297
The loss at :  3  is :  456483.6765136719
The loss at :  4  is :  314745.21252441406
The loss at :  5  is :  333939.6365966797
The loss at :  6  is :  309251.72216796875
The loss at :  7  is :  270435.88818359375
The loss at :  8  is :  274496.134765625
The loss at :  9  is :  292545.06103515625
The loss at :  10  is :  325156.9680175781
The loss at :  11  is :  332082.5089111328
The loss at :  12  is :  391501.3350830078
The loss at :  13  is :  265278.98376464844
The loss at :  14  is :  240099.63623046875
The loss at :  15  is :  280924.58544921875
The loss at :  16  is :  290306.08447265625
The loss at :  17  is :  255689.6075439453
The loss at :  18  is :  298363.1787109375
The loss at :  19  is :  330333.7375488281
The loss at :  20  is :  285181.59326171875
The loss at :  21  is :  244514.4315185547
The loss at :  22  is :  334025.15100097656
The loss at :

In [21]:
# Encode and decode the digit image
encoded_train = sess.run(encoder_op, feed_dict={X: X_train})
encoded_test = sess.run(encoder_op, feed_dict={X: X_test})


In [54]:
from sklearn.metrics.pairwise import cosine_similarity
old_distance = cosine_similarity(X_test)
new_distance = cosine_similarity(encoded_test)

In [56]:
for i in range(X_test.shape[0]):
    old = np.argsort(-old_distance[i])
    new = np.argsort(-new_distance[i])
    print("old:",old[:20])
    print("new:",new[:20])

old: [  0 444 323  69 282  52  57  61 127 184 201 272 419 106 393 430 445 187
  23 144]
new: [  0 323 444 367  65 282  69  52 235 106 451 102  15 389 233 159 299  57
 116 390]
old: [  1 317  62  81 333 105 158 441 121 175  25  21 423   2  14 382 294 369
 143  89]
new: [  1 358 211 249  70 424 289 449   9 190 422 445 128 184 312 103 371 359
 155 234]
old: [  2 333 382 298 423 233  14  51 369  39 163 190 177 251 196  53 327 325
 434 416]
new: [  2 186 290 196  43 298 154 316 231 142 376 297 331 218 390 448 277 233
 181 148]
old: [  3  10 332 131 239 460 342 124 349 114 111 433  31 400 409  47  87  88
 438 426]
new: [  3 113 173 392 210 110  94 243 140 427 428  89  55 436 103 371 442 368
   8 222]
old: [  4  63 281 398 195 306 103 326 450  99  44  78 120 404 381 258  72 421
 355 362]
new: [  4  48 188 375 121 146  56 423 142 406 450   8 244 171 377 441  51 298
 210 175]
old: [  5 172 138 452 378 194 389 439 255 456 256 351 182 388 401 273 330 193
 446 166]
new: [  5 138 334 413  60  71 38

old: [445 392 409 460 187 291 144 239 131  31 332  16  88 459 426 106 128 278
 124  87]
new: [445 289 128 453 449 308 184 149 431 358 370 249 458 108 380  17 457 169
 211 319]
old: [446 183 263 351   8 283 244 396 240 193 441 173 161  81 456  34 324 153
 336 317]
new: [446 353 183 330 197  75 286  11 206 204 133 440 350 288 420  85 123 292
 165 455]
old: [447 153 283 318  73 446 173 330 303 351 263 439 193 350  34 213   8 456
 443 248]
new: [447 248 416 214 345 223 172 293 195 351 275 350 283 291 368  89  16 305
 284 302]
old: [448  44 374 371 450 103  93 281 422  92 316 270 326   4 365 343 195 352
 383  75]
new: [448 374 175 234 298 423 406 186   2 356 142 441 450 290 277 171  48  51
 210 196]
old: [449 144 445 291 460 409 392 128 131 332  31 239 187 426 459  88 262  16
 278  87]
new: [449 453 445 128 289 249 457 169 149 184 431 358 308 458  17 211 370 319
 108 380]
old: [450 103 281 371 316  44 195 326  99 230 292 422 352 120 343 258 381 421
 357 404]
new: [450 441 210 277 423  51 17

In [51]:
encoded_train.shape

(4140, 5)

In [52]:
## Import the Classifier.
#K = 5
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=5)
## Fit the model on the training data.
knn.fit(encoded_train, y_train)
## See how the model performs on the test data.
knn.score(encoded_test, y_test)

0.89587852494577

In [12]:
#K = 3
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=5)
## Fit the model on the training data.
knn.fit(encoded_train, y_train)
## See how the model performs on the test data.
knn.score(encoded_test, y_test)

0.8806941431670282

In [22]:
#K = 4
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=5)
## Fit the model on the training data.
knn.fit(encoded_train, y_train)
## See how the model performs on the test data.
knn.score(encoded_test, y_test)

0.8915401301518439

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression(penalty='l2', random_state=42)
logreg.fit(encoded_train, y_train)
y_pred = logreg.predict(encoded_test)
print("accuracy",metrics.accuracy_score(y_test, y_pred))

accuracy 0.8698481561822126
